# 데이터 이해하기, 어떻게 분석할지 고민하기, 준비하기

In [ ]:
!samtools view ../datapack/CLIP-35L33G.bam | head -6

In [ ]:
# 전체 read 개수
!samtools view ../datapack/CLIP-35L33G.bam | wc -l

* GSNAP 깃헙 readme 설명 중 sam output 에 대한 설명을 읽어보면 `XS:` tag가 splice의 strand orient를 표기한다. splice donor-acceptor 페어가 +/- strand 중 어떤 strand에 매치되는지를 표기하는 듯 하다. splice가 모호하거나 가능성이 낮다고 판단되면 XS:A:+ 혹은 XS:A:-로 표기한다고 나와있다. 일단 새로 alignment를 진행하는 일 없이 `CLIP-35L22G.bam`파일에서 작업을 시작하고 싶으므로, 이 XStag를 사용하여 분석을 진행한다.  

In [ ]:
# splicing을 포함하는 read의 경우는 XStag가 어떻게 표현되는 지.
!samtools view ../datapack/CLIP-35L33G.bam | awk '$6 ~/N/ {print $0}' | grep "XS:A:+" | head -3

* splicing 있는 read도 그냥 XS:A:?로 표시된다.
* 그럼 그냥 XS:A:로 strand구별하면 될까

In [ ]:
# 모든 read가 XStag를 가지고 있을까.
# 그리고 XS:A:+/-는 각각 몇개씩
!samtools view ../datapack/CLIP-35L33G.bam | grep -v 'XS:' | wc -l #XS없는 read
!samtools view ../datapack/CLIP-35L33G.bam | grep 'XS:A:+' | wc -l 
!samtools view ../datapack/CLIP-35L33G.bam | grep 'XS:A:-' | wc -l

In [ ]:
(16152362 + 14573778)/ (38880853-8154713) # XS달린 read 중 XS:A:+ 혹은 XS:A:-의 비율

* 결국 이 bam파일에서는 spliced read이거나 아니거나 관계 없이 XS:A:? 태그로 strand orientation을 구분할 수 있음을 알 수 있었다. +strand만으로도 개수가 꽤 많으니 일단 +만 가지고 분석을 진행한다.
* 일단 `MD:Z:`부분이 mismatch를 표시하고 있기 때문에, 이 부분의 정보를 가지고 read 위치별 error frequency를 구하는 방법으로 진행한다.
1. "+ strand" aligned read의 cigar, read length, MDtag만 추출한다.
2. 1을 이용하여 5'end쪽의 softclip 에 해당하는 error는 제외한 position(20bins)별 에러 정보를 축적한다.
3. 2를 가지고 error frequency 그래프를 error type별로 그린다 (Fig2C).
4. 2를 가지고 base-specific error frequency 그래프를 error type별로 그린다 (Fig2D)

# 1. Extract CIGAR, read length, MD from bam

*일단 XS:A:+인 reads의 cigar ($6), seq length ($10), 그리고 bam의 끝의 metadata부분의 첫번째 열만 일단 가져와본다 (gawk의 gensub 이나 기타 여러가지 다른 방법으로 MD를 다 추출해보려고했는데 계속 잘 안되어서 할수 없이...)
```{python}
!samtools view ../datapack/CLIP-35L33G.bam | grep "XS:A:+" | awk '{print $6"\t"length($10)"\t"$12}' > temp.txt
```

In [ ]:
!wc -l temp.txt
!grep "MD:Z:" temp.txt | wc -l

* 아마도 어떤 라인은 MD태그가 12번 열이 아닌 다른 열에 위치하기도 하는 모양이다...
* 근데 이 bam파일의 metadata에서의 구분자도 \t이라서 통째로 추출이 어렵다.
* 그냥 몇개 안되니까 MD태그 안잡힌 리드 정보들은 그냥 제외하고 진행한다.

In [ ]:
!grep "MD:Z:" temp.txt > CigarLenMD_plus.txt
!wc -l CigarLenMD_plus.txt

# 2. Pileup Errors across Reads

In [ ]:
import re
def Err_eachMDflag(mdflag):
    p = re.compile("[0-9]+[\^]*[ACGT]+")
    
    temp=[]
    errorI = 0
    for err in p.findall(mdflag):
        matchN = int(re.search("[0-9]+",err).group())
        errorI += matchN 
        errorB = re.search("[\^ACGT]+", err).group()
        temp.append((errorI, errorB))
        if errorB.startswith("^"):
            errorI -= 1
    return temp

In [ ]:
test = Err_eachMDflag("MD:Z:15C15G21")
test

* 작성한 함수를 활용하여 결과리스트를 도출하는 과정 테스트

In [ ]:
test = ["CIGAR\t31\t18^C13", "CIGAR2\t28\t28", "CIGAR3\t51\t15C0G14G21"]

testlist =[]
for i in range(20):
    testlist.append([])
    
    
for line  in test:
    length = int(line.split("\t")[1])
    md = line.split("\t")[2]
    binsize = length//20
    errList = Err_eachMDflag(md)
    for i, b in errList:
        testlist[i//binsize].append(b)

testlist

* 본 작업

In [ ]:
import time
start = time.time()

reslist =[]
for i in range(40): # 나머지를 위해 리스트 길이를 40으로 만들고 결과를 보고 앞뒤를 trim한다.
    reslist.append([])

f = open("CigarLenMD_plus.txt", "r")
nline = len(f.readlines())
f.seek(0)
for i in range(nline):
    cigar, readlen, md = f.readline().split("\t")
    softclip = re.findall("[0-9]*S", cigar)
    if softclip:
        softclip5end = int(softclip[0][:-1]) #drop "S" and convert int
    else:
        pass
    binsize = int(readlen)//20
    errList = Err_eachMDflag(md)
    for i, b in errList:
        if i <= softclip5end : #considering only softclip at 5'end
            pass
        else:
            reslist[i//binsize].append(b)
f.close()

print("time: ", time.time() - start)

In [ ]:
import numpy as np
np.array([len(eachpos) for eachpos in reslist])

In [ ]:
import matplotlib.pyplot as plt

x = list(range(40))
y1 = np.array([len(eachpos) for eachpos in reslist])/nline
plt.scatter(x, y1)	
plt.grid(True)
plt.show()	

* 한 3번째 부터 포함해서  20개 bin 이후에 있는 error들은 무시해도 될 듯 하다

In [ ]:
reslist_cut = reslist[3:23]

# 3. Error Frequency Graph (FigS2C)

In [ ]:
from PIL import Image
img1 = Image.open("../goal_myproject.png")
dim1 = (0,0,650, 720)
crop_img1 = img1.crop(dim1)
crop_img1

In [ ]:
DelErrCount = [sum(1 if eacherr.startswith("^") else 0 for eacherr in errPos) \
               for errPos in reslist_cut]
SubErrCount = [sum(0 if eacherr.startswith("^") else 1 for eacherr in errPos) \
               for errPos in reslist_cut]

In [ ]:
SubErrFreq =  np.array(SubErrCount)/nline
DelErrFreq =  np.array(DelErrCount)/nline

In [ ]:
x = list(range(20))
y1 = SubErrFreq*100
y2 = DelErrFreq*100
plt.figure(figsize=(6,6))
plt.title("Error Frequency across CLIP-seq reads\n(35L33G)", size = 15)
plt.xlabel("Position in tag", size = 14)
plt.ylabel("Frequency (percent)", size = 14)
plt.plot(x, y1, "#E19323", label= "Substitution", linewidth ="3")	
plt.plot(x, y2, "#233C74", label = "Deletion", linewidth = "3")	
plt.legend(loc = "upper right")
plt.xticks([0, 5, 10, 15, 20])
plt.tick_params(axis='y', direction='in', length=3, pad=6, labelsize=12)
plt.tick_params(axis='x', direction='in', length=3, pad=6, labelsize=0)
plt.grid(linestyle='--',alpha = 0.8,linewidth="1")
plt.show()	

* 비슷한듯 다른 그래프....이지만 일단 이대로 만족하고
* error type별로 base-specific freqeucney도 뽑아보자

# 4. Base-specific Error Frequency Graphs (FigS2D)

## 4-1. Substitution

In [ ]:
start = time.time()
#리스트컴프리헨션, if함수, join함수 그리고 count한번에 쓰려면 시간이 영원히 걸리더라.
#그래서 일단 이렇게 나눠서 진행
SubErrlist = [[eacherr for eacherr in errPos if not eacherr.startswith("^")] \
              for errPos in reslist_cut]
DelErrlist = [[eacherr for eacherr in errPos if eacherr.startswith("^")] \
              for errPos in reslist_cut]
print(time.time()- start)

In [ ]:
start = time.time()

SubErrAcount = np.zeros(shape=(20,), dtype=np.int64)
SubErrCcount = np.zeros(shape=(20,), dtype=np.int64)
SubErrGcount = np.zeros(shape=(20,), dtype=np.int64)
SubErrTcount = np.zeros(shape=(20,), dtype=np.int64)
for i, errPos in enumerate(SubErrlist):
    SubErrAcount[i] = sum(eachErr.count("A") for eachErr in errPos)
    SubErrCcount[i] = sum(eachErr.count("C") for eachErr in errPos)
    SubErrGcount[i] = sum(eachErr.count("G") for eachErr in errPos)
    SubErrTcount[i] = sum(eachErr.count("T") for eachErr in errPos)
print("time: ", time.time() - start)

In [ ]:
SubErrAfreq = SubErrAcount/nline
SubErrCfreq = SubErrCcount/nline
SubErrGfreq = SubErrGcount/nline
SubErrTfreq = SubErrTcount/nline

In [ ]:
x = list(range(20))
y1 = SubErrAfreq*100
y2 = SubErrCfreq*100
y3 = SubErrGfreq*100
y4 = SubErrTfreq*100
plt.figure(figsize=(6,6))
plt.title("Base-Specific Error Frequency across CLIP-seq reads\n(35L33G, Substitution)", size = 15)
plt.xlabel("Position in tag", size = 14)
plt.ylabel("Frequency (percent)", size = 14)
plt.plot(x, y1, "#233C74", label = "A")	
plt.plot(x, y2, "#4C842C", label = "C") 
plt.plot(x, y3, "#E19323", label = "G")
plt.plot(x, y4, "#AB111B", label = "U")
plt.legend(loc = "upper right")
plt.xticks([0, 5, 10, 15, 20])
plt.tick_params(axis='y', direction='in', length=3, pad=6, labelsize=12)
plt.tick_params(axis='x', direction='in', length=3, pad=6, labelsize=0)
plt.grid(linestyle='--',alpha = 0.8,linewidth="1")
plt.show()	

# 4-2. Deletion

In [ ]:
start = time.time()
DelErrAcount = np.zeros(shape=(20,), dtype=np.int64)
DelErrCcount = np.zeros(shape=(20,), dtype=np.int64)
DelErrGcount = np.zeros(shape=(20,), dtype=np.int64)
DelErrTcount = np.zeros(shape=(20,), dtype=np.int64)
for i, errPos in enumerate(DelErrlist):
    DelErrAcount[i] = sum(eachErr.count("A") for eachErr in errPos)
    DelErrCcount[i] = sum(eachErr.count("C") for eachErr in errPos)
    DelErrGcount[i] = sum(eachErr.count("G") for eachErr in errPos)
    DelErrTcount[i] = sum(eachErr.count("T") for eachErr in errPos)
print("time: ", time.time() - start)

In [ ]:
DelErrAfreq = DelErrAcount/nline
DelErrCfreq = DelErrCcount/nline
DelErrGfreq = DelErrGcount/nline
DelErrTfreq = DelErrTcount/nline

In [ ]:
x = list(range(20))
y1 = DelErrAfreq*100
y2 = DelErrCfreq*100
y3 = DelErrGfreq*100
y4 = DelErrTfreq*100
plt.figure(figsize=(6,6))
plt.title("Base-Specific Error Frequency across CLIP-seq reads\n(35L33G, Deletion)", size = 15)
plt.xlabel("Position in tag", size = 14)
plt.ylabel("Frequency (percent)", size = 14)
plt.plot(x, y1, "#233C74", label = "A")	
plt.plot(x, y2, "#4C842C", label = "C") 
plt.plot(x, y3, "#E19323", label = "G")
plt.plot(x, y4, "#AB111B", label = "U")
plt.legend(loc = "upper right")
plt.xticks([0, 5, 10, 15, 20])
plt.tick_params(axis='y', direction='in', length=3, pad=6, labelsize=12)
plt.tick_params(axis='x', direction='in', length=3, pad=6, labelsize=0)
plt.grid(linestyle='--',alpha = 0.8,linewidth="1")
plt.show()	

## Think more

* 이번에는 insertion 에러는 다루지 않고 각 error의 read상에서 postion을 파일업 할 때도 insertion은 무시하였다. 시작 포인트로 사용한 bam파일에서 Insertion error의 base정보를 알수가 없었기 때문이다. 그러나 이 insertion 어느정도로 많은지, 정말 무시해도 될 정도인지는 확인하는게 좋을 것같다.

In [ ]:
!samtools view ../datapack/CLIP-35L33G.bam | cut -f 6 | grep "I" | wc -l

In [ ]:
print(80873/38880853)

* sam파일 매뉴얼에 없는 MD tag가 발견되어, 이렇게 MD가 표기된 read의 다른 정보들은 어떨지 잠깐 살펴본다.

In [ ]:
!samtools view ../datapack/CLIP-35L33G.bam | grep "MD:Z:0CTG" | head -6

* 위 결과를 보면 CIGAR가 40M이니 다른 종류의 error는 아닌 것 같다. 그리고 `MD:Z:0CTG37`이랑 `NM:i:3`을 보니까 `MD:Z:0C0T0G37`이랑 동일한 상황인거같다.
* 아니면 혹시 이 alignmnet에서는 연속으로 substitution error가 발생했을 때,`[0-9]+[ACTG]0[ACTG]`같은 식의 표현을 사용하지 않고 `[0-9]+[ACTG]+`로 표시하는 걸까 확인해봤는데 그렇지도 않다.

In [ ]:
!samtools view ../datapack/CLIP-35L33G.bam | grep "MD:Z:.*[0-9][ACGT][0][ACGT]" | head -3